### Pandas group 연산자
* 분류 기준
    * 내부의 연산 매서드
    * 원하는 연산 매서드가 없을 때 함수를 별도로 제작하여 연산결과를 출력하게 돕는 매서드 
    * 조건에 부합하는 데이터프레임을 출력하도록 돕는 매서드 
* groupbyresult.groups
* df2.data1.groupby(df2.key1).sum()
* df2.groupby(df2.key1)["data1"].sum()
* df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2") # 피벗테이블 형태로
* agg : 함수 제작 후 agg 메서드에 전달하면 "그룹 연산"을 진행한다.
    def peak_to_peak_ratio(x):  
        return x.max() / x.min()  
    iris.groupby(iris.species).agg(peak_to_peak_ratio)  
* apply : 원하는 대표 연산이 없을 때, "데이터 프레임을 출력"하기 위해 사용함
    def top3_petal_length(df):    
        return df.sort_values(by="petal_length", ascending=False)[:3]    
    iris.groupby(iris.species).apply(top3_petal_length)  
* row 합계= df.sum(axis = 1)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data_preprocessing/tf_seoul/tf_seoul_200617_after_jipgyegu.csv')

/Users/yeonjin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
height = pd.DataFrame(df.groupby(['법정동코드_구', '법정동구', '지형높이'])['기준월'].count()).reset_index()

In [30]:
height_sum = pd.DataFrame(height.groupby(['법정동코드_구', '법정동구'])['기준월'].sum()).reset_index()

In [31]:
height_sum.columns = ['법정동코드_구', '법정동구', '지형높이sum']

In [36]:
height_sum.head()

,법정동코드_구,법정동구,지형높이sum
0,11110,서울특별시 종로구,50760
1,11140,서울특별시 중구,38174
2,11170,서울특별시 용산구,40227
3,11200,서울특별시 성동구,35169
4,11215,서울특별시 광진구,34220


In [35]:
height_agg = pd.merge(height, height_sum,  how = 'left')
height_agg.head()

,법정동코드_구,법정동구,지형높이,기준월,지형높이sum
0,11110,서울특별시 종로구,고지,3341,50760
1,11110,서울특별시 종로구,급경사,5689,50760
2,11110,서울특별시 종로구,완경사,8204,50760
3,11110,서울특별시 종로구,저지,22,50760
4,11110,서울특별시 종로구,지정되지않음,7816,50760


In [38]:
height_agg.columns

Index(['법정동코드_구', '법정동구', '지형높이', '기준월', '지형높이sum'], dtype='object')

In [39]:
height_agg.columns = ['법정동코드_구', '법정동구', '지형높이', '지형높이count', '지형높이sum']

In [41]:
height_agg['percent'] = round(height_agg['지형높이count'] / height_agg['지형높이sum'], 3)

In [105]:
height_agg.head()

,법정동코드_구,법정동구,지형높이,지형높이count,지형높이sum,percent
0,11110,서울특별시 종로구,고지,3341,50760,0.066
1,11110,서울특별시 종로구,급경사,5689,50760,0.112
2,11110,서울특별시 종로구,완경사,8204,50760,0.162
3,11110,서울특별시 종로구,저지,22,50760,0.000
4,11110,서울특별시 종로구,지정되지않음,7816,50760,0.154


In [106]:
height_agg.groupby(['법정동코드_구', '법정동구'])['지형높이','percent'].max().sort_values('percent').head()

,,지형높이,percent
법정동코드_구,법정동구,,
11590,서울특별시 동작구,평지,0.388
11410,서울특별시 서대문구,평지,0.459
11620,서울특별시 관악구,평지,0.475
11110,서울특별시 종로구,평지,0.506
11380,서울특별시 은평구,평지,0.526


In [47]:
# len(height_agg.groupby(['법정동코드_구', '법정동구'])['지형높이','percent'].max())
# 25개구

25

In [107]:
K = height_agg.pivot_table("percent", ['법정동코드_구', '법정동구'], '지형높이')
K.head()

,지형높이,고지,급경사,완경사,저지,지정되지않음,평지
법정동코드_구,법정동구,,,,,,
11110,서울특별시 종로구,0.066,0.112,0.162,0.000,0.154,0.506
11140,서울특별시 중구,0.000,0.000,0.131,0.000,0.242,0.626
11170,서울특별시 용산구,0.000,0.000,0.540,0.003,0.003,0.454
11200,서울특별시 성동구,0.024,0.002,0.184,0.010,0.078,0.702
11215,서울특별시 광진구,0.000,0.004,0.062,0.001,0.191,0.742


In [61]:
K.reset_index(inplace = True)

In [51]:
#K.to_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data_preprocessing/tf_seoul/tf_seoul_200618_height.csv')

In [109]:
#2. 산지를 제외하고 공시지가, 토지면적 
sum_s = pd.DataFrame(df[df['대장구분명']!='산'].groupby(['법정동코드_구', '법정동구'])[['공시지가','토지면적']].sum()).reset_index()
sum_s.head()

,법정동코드_구,법정동구,공시지가,토지면적
0,11110,서울특별시 종로구,276134754200,15153393.5
1,11140,서울특별시 중구,324442496300,8840802.8
2,11170,서울특별시 용산구,236185543300,14541295.3
3,11200,서울특별시 성동구,128992752900,17637684.3
4,11215,서울특별시 광진구,118282995100,17992140.4


In [72]:
sum_s['면적당가격'] = round(sum_s['공시지가'] / sum_s['토지면적'],2)

In [73]:
merged_gu = pd.merge(K, sum_s)

In [76]:
#merged_gu.to_csv('/Users/yeonjin/Documents/GitHub/Market-Entry-Analysis/topography_data_preprocessing/tf_seoul/topology_200618.csv')

In [88]:
# 3. 지형형상
ss = pd.DataFrame(df.groupby(['법정동코드_구', '법정동구', '지형형상'])['지형형상코드'].count()).reset_index()
ss.columns = ['법정동코드_구', '법정동구', '지형형상', '지형형상count']

In [96]:
ss

,법정동코드_구,법정동구,지형형상,지형형상count
0,11110,서울특별시 종로구,가로장방,2458
1,11110,서울특별시 종로구,부정형,12928
2,11110,서울특별시 종로구,사다리형,18884
3,11110,서울특별시 종로구,삼각형,997
4,11110,서울특별시 종로구,세로장방,4248
...,...,...,...,...
220,11740,서울특별시 강동구,세로장방,6033
221,11740,서울특별시 강동구,역삼각형,29
222,11740,서울특별시 강동구,자루형,1262
223,11740,서울특별시 강동구,정방형,3377


In [97]:
ss2 = pd.DataFrame(df.groupby(['법정동코드_구', '법정동구'])['지형형상코드'].count()).reset_index()
ss2.columns = ['법정동코드_구', '법정동구', '지형형상sum']

In [99]:
ss3 = pd.merge(ss,ss2, how='left')
ss3['percent'] = round(ss3['지형형상count'] / ss3['지형형상sum'],3)

In [100]:
ss3

,법정동코드_구,법정동구,지형형상,지형형상count,지형형상sum,percent
0,11110,서울특별시 종로구,가로장방,2458,50760,0.048
1,11110,서울특별시 종로구,부정형,12928,50760,0.255
2,11110,서울특별시 종로구,사다리형,18884,50760,0.372
3,11110,서울특별시 종로구,삼각형,997,50760,0.020
4,11110,서울특별시 종로구,세로장방,4248,50760,0.084
...,...,...,...,...,...,...
220,11740,서울특별시 강동구,세로장방,6033,31643,0.191
221,11740,서울특별시 강동구,역삼각형,29,31643,0.001
222,11740,서울특별시 강동구,자루형,1262,31643,0.040
223,11740,서울특별시 강동구,정방형,3377,31643,0.107


In [101]:
ndf = ss3.pivot_table('percent',['법정동코드_구', '법정동구'], '지형형상' )

In [103]:
ndf.reset_index(inplace = True)

In [110]:
ndf.head()

지형형상,법정동코드_구,법정동구,가로장방,부정형,사다리형,삼각형,세로장방,역삼각형,자루형,정방형,지정되지않음
0,11110,서울특별시 종로구,0.048,0.255,0.372,0.020,0.084,0.001,0.026,0.029,0.167
1,11140,서울특별시 중구,0.061,0.019,0.465,0.011,0.135,0.000,0.034,0.030,0.245
2,11170,서울특별시 용산구,0.066,0.264,0.443,0.026,0.124,0.000,0.022,0.051,0.003
3,11200,서울특별시 성동구,0.056,0.179,0.347,0.013,0.104,0.000,0.021,0.076,0.205
4,11215,서울특별시 광진구,0.128,0.086,0.163,0.011,0.234,0.000,0.045,0.141,0.192
